In [78]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import nltk
import string
from string import digits
from langdetect import detect

In [41]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
URL="http://www.20000-names.com/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

Get all linls from homepage from central table

In [46]:
links={}
for a in soup.find_all('td'):
    for b in a.find_all('a', href=True):

        if("Boy" in b.text or "Girl"in b.text) :
            links.update({URL+b.get("href"): b.text[b.text.find('')+len(''):b.text.rfind('\r\n\r')]})

            for i in range(2,21):

                if requests.get(URL+b.get("href")[:-4]+'_0'+str(i)+'.htm').status_code==200:
                        links.update({URL+b.get("href")[:-4]+'_0'+str(i)+'.htm': b.text[b.text.find('')+len(''):b.text.rfind('\r\n\r')]})

                elif requests.get(URL+b.get("href")[:-4]+'_'+str(i)+'.htm').status_code==200:
                    links.update({URL+b.get("href")[:-4]+'_'+str(i)+'.htm': b.text[b.text.find('')+len(''):b.text.rfind('\r\n\r')]})

                else:
                    break

In [48]:
gender=[]
location=[]
names=[]
meaning=[]
for URL in links.keys():
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    for b in soup.find_all('li'):
        if(b.text!=''):
            all_text=("".join(b.text.rsplit("\r\n\r\n")))
            tokens = nltk.word_tokenize(all_text)
            names.append(tokens[0])
            meaning.append(" ".join(tokens[1:]))
            gender.append(URL[URL.find('.com/')+len('.com/'):URL.find('_')])
            location.append(URL[URL.find('_')+len('_'):URL.rfind('_')])

In [49]:
other_names=[]
other_lang_m=[]
loc=[]
for i in range(len(meaning)):
    if meaning[i][0]=='(':
        #other_names.append(names[i])
        a=meaning[i][meaning[i].find('(')+len('('):meaning[i].find(')')]
        a = ''.join([s for s in a if not s.isdigit()])
        a = ''.join([s for s in a if s !=':'])
        a = ''.join([s for s in a if s !=','])
        
        if("pronounced" not in nltk.word_tokenize(a) and "pron" not in nltk.word_tokenize(a) and "Pronounced" not in nltk.word_tokenize(a)):
            other_names.append(names[i])
            other_lang_m.append(a)
            loc.append(location[i])

In [90]:
or_n=[]
ot_n=[]
l=[]
lang=[]
count=0
for i in range(len(other_lang_m)):
    tokens = nltk.word_tokenize(other_lang_m[i])
    if(len(tokens)==1):
        
        or_n.append(other_names[i])
        ot_n.append(tokens[0])
        lang.append(loc[i])
        l.append(loc[i])

    elif(len(tokens)==2 and tokens[0][0].isalpha() and tokens[0][0].isupper() and tokens[0][0]in list(string.ascii_letters)):
        
        count+=1
        or_n.append(other_names[i])
        ot_n.append(tokens[1])
        lang.append(tokens[0])
        l.append(loc[i])
    elif(len(tokens)==2 ):
        
        or_n.append(other_names[i])
        if(tokens[0][0]=='-'):
            ot_n.append(tokens[0][1:])
        else:
            ot_n.append(tokens[0])
        lang.append(loc[i])
        l.append(loc[i])
        
        or_n.append(other_names[i])
        if(tokens[1][0]=='-'):
            ot_n.append(tokens[1][1:])
        else:
            ot_n.append(tokens[1])
        lang.append(loc[i])
        l.append(loc[i])
    elif(len(tokens)==4 and  tokens[0][0].isalpha() and len(tokens[0])>1 and 'and' not in tokens and tokens[0][0].isupper() ):
        
        for s in range(0,len(tokens),2):
            or_n.append(other_names[i])
            ot_n.append(tokens[s+1])
            lang.append(tokens[s])
            l.append(loc[i])
    elif('and' in tokens):
            
            or_n.append(other_names[i])
            ot_n.append(tokens[3])
            lang.append(tokens[2])
            l.append(loc[i])
            or_n.append(other_names[i])
            ot_n.append(tokens[3])
            lang.append(tokens[0])
            l.append(loc[i])
    elif('or' in tokens):
            or_n.append(other_names[i])
            ot_n.append(tokens[0])
            lang.append(loc[i])
            l.append(loc[i])
            or_n.append(other_names[i])
            ot_n.append(tokens[2])
            lang.append(loc[i])
            l.append(loc[i])
    elif(len(tokens)==6 and tokens[0]!="Chinese" and tokens[0][0]!="-"):
        
        for s in range(0,len(tokens),2):
            or_n.append(other_names[i])
            ot_n.append(tokens[s+1])
            lang.append(tokens[s])
            l.append(loc[i])
    elif(len(tokens)==6 and tokens[0]=="Chinese"):
            
            or_n.append(other_names[i])
            ot_n.append(tokens[1])
            lang.append(tokens[0])
            l.append(loc[i])
            
            or_n.append(other_names[i])
            ot_n.append(tokens[2])
            lang.append(tokens[0])
            l.append(loc[i])
            
            or_n.append(other_names[i])
            ot_n.append(tokens[4])
            lang.append(tokens[3])
            l.append(loc[i])
            
            or_n.append(other_names[i])
            ot_n.append(tokens[5])
            lang.append(tokens[3])
            l.append(loc[i])
            
            
    # elif(not tokens[0][0].isalpha()):
    #     #print(tokens)
    #     for s in range(len(tokens)):
    #         or_n.append(other_names[i])
    #         if(tokens[s][0]=='-'):
    #             ot_n.append(tokens[s][1:])
    #         else:
    #             ot_n.append(tokens[s])
    #         lang.append(loc[i])
    #         l.append(loc[i])
        
    else:
        #print(tokens)
        if(tokens[0]!='f'):
            for s in range(len(tokens)):
              or_n.append(other_names[i])
              if(tokens[s][0]=='-'):
                  ot_n.append(tokens[s][1:])
              else:
                  ot_n.append(tokens[s])
              lang.append(loc[i])
              l.append(loc[i])


In [91]:
for i in range(len(lang)):
    if '_' in lang[i] and 'names' in lang[i]:
        lang[i]=lang[i][:lang[i].rfind('_')]
        lang[i]=lang[i].capitalize()
    else:
        lang[i]=lang[i].capitalize()

for i in range(len(l)):
    if '_' in l[i] and 'names' in l[i]:
        l[i]=l[i][:l[i].rfind('_')]
        l[i]=l[i].capitalize()
    else:
        l[i]=l[i].capitalize()

In [107]:
# df = pd.DataFrame({'Original Name':or_n,
#                    'Name in Other Language': ot_n,
#                  'Language':lang,
#                   'Location':l
#                   })
# #df
# df.to_csv(index=False)
# compression_opts = dict(method='zip',
#                         archive_name='names_other_languages.csv')  
# df.to_csv('/content/drive/MyDrive/Colab Notebooks/Analyse/names_other_languages.zip', index=False,
#           compression=compression_opts) 

In [103]:
for i in range(len(meaning)):
    if(meaning[i][0]==',' or meaning[i][0]==':' or meaning[i][0]==' '):
        meaning[i]=meaning[i][1:]
        meaning[i]=meaning[i].strip()
        meaning[i]=meaning[i].strip('.')
        meaning[i]=meaning[i].strip()
        meaning[i]=meaning[i].strip('.')
    elif(meaning[i][0]=='('):
        #print(meaning[i])
        meaning[i]=meaning[i][meaning[i].find(')')+len(')')+2:]
        meaning[i]=meaning[i].strip()
        meaning[i]=meaning[i].strip('.')
        meaning[i]=meaning[i].strip()
        meaning[i]=meaning[i].strip('.')


        
for i in range(len(location)):
    if '_' in location[i] and 'names' in location[i]:
        location[i]=location[i][:location[i].rfind('_')]
        location[i]=location[i].capitalize()
    else:
        location[i]=location[i].capitalize()


In [108]:
# df1 = pd.DataFrame({ 'Original Name':names,
#                      'Gender': gender,
#                      'Description':meaning,
#                      'Location':location
#                   })
# #df
# df1.to_csv(index=False)
# compression_opts = dict(method='zip',
#                         archive_name='name_descriptions.csv')  
# df1.to_csv('/content/drive/MyDrive/Colab Notebooks/Analyse/name_descriptions.zip', index=False,
#           compression=compression_opts) 